In [2]:
import pandas as pd

In [3]:
eval_df = pd.read_json("predictions_checkpoint_1800.json")

In [4]:
eval_df.head()

,company_id,year,actual_label,predicted_direction,magnitude,certainty,reason,is_match,full_model_response
0,1000,1973,increase,decrease,Significant,Medium,\eSince\e\ sales\ehave\edeclined\esharply\ewit...,False,system\nYou are a helpful assistant.\nuser\nBa...
1,1000,1974,decrease,decrease,Moderate,Medium,"Despite some growth signals via asset buildup,...",True,system\nYou are a helpful assistant.\nuser\nBa...
2,1000,1975,increase,increase,Moderate,High,Rapidly increasing sales coupled with margin e...,True,system\nYou are a helpful assistant.\nuser\nBa...
3,1000,1976,increase,decrease,Moderate,Medium,\nThe projected decrease in EPS stems from ant...,False,system\nYou are a helpful assistant.\nuser\nBa...
4,1000,1977,decrease,error,Error,Error,"Error in parsing JSON: Expecting ',' delimiter...",False,system\nYou are a helpful assistant.\nuser\nBa...


In [5]:
eval_df["full_model_response"][0]

'system\nYou are a helpful assistant.\nuser\nBased on the following financial statements, assess whether EPS will increase or decrease in the next year:\n\nBelow is the Balance Sheet:\n------------------------------------------------------------------------------------------\n| Account Items                                                  | 1971 | 1972 |\n------------------------------------------------------------------------------------------\n| Total Asset                                                  | 29.330 | 19.907 |\n| Current Assets                                               | 19.688 | 11.326 |\n| Current Liabilities                                          | 8.507 | 3.990 |\n| Cash and Short-Term Investments                              | 2.557 | 2.027 |\n| Receivables                                                  | 8.398 | 4.186 |\n| Inventories                                                  | 8.187 | 4.743 |\n| Other Current Assets                               

In [ ]:
eval_df.predicted_direction.value_counts()
# len(eval_df)
957

predicted_direction
error                                   560
increase                                559
decrease                                398
<increase/decrease>                     122
moderate increase                        40
uncertain                                39
possible increase                        24
possible decrease                        12
moderate                                  8
slight increase                           6
steady                                    4
stable                                    3
stabilize                                 2
moderate decrease                         2
decreased                                 2
possible stabilization                    1
possibly decrease                         1
stabilization                             1
unknown                                   1
maintained                                1
decremented                               1
increased                                 1
slight      

In [7]:
full_text = eval_df[eval_df.predicted_direction == "error"].full_model_response.iloc[0]

In [69]:
text = full_text.split('assistant')[-1]
# print(text)

In [9]:
import re
import json

pattern = r'```json\s*\n(.*?)\n```'
    
# Find all JSON blocks
json_blocks = re.findall(pattern, text, re.DOTALL)[0]
print(json_blocks)

{
    "direction_of_eps_change": "Moderate Increase",
    "magnitude_of_change": "Dependent on successful execution of growth initiatives while optimizing cost structures and managing existing debts.",
    "certainty_of_assessment": "Medium",
    "reason": "\\nThe company’s demonstrated ability to grow revenues with expanding markets alignss with upward pressure on earnings per share. However, the balance between financial leverage and operational efficiency remains critical, affecting both the extent of such increase and its predictability."
"
}


In [10]:
from json_repair import repair_json
import json

def fix_json_with_library(broken_json_text):
    """
    Use json-repair library to fix common JSON issues.
    """
    try:
        # Attempt to repair the JSON
        fixed_json = repair_json(broken_json_text)
        return json.loads(fixed_json)
    except Exception as e:
        print(f"Could not repair JSON: {e}")
        return None

In [11]:
fix_json_with_library(json_blocks)['direction_of_eps_change']

'Moderate Increase'

It is working...

In [70]:
def fixed_json(df):
    all_fixed_jsons = []

    for i, row in df.iterrows():
        is_error = (row.predicted_direction == "error")
        if not is_error:
            continue
        
        full_response_text = row.full_model_response
        last_text = full_response_text.split('assistant')[-1]

        # Try primary pattern (```json ... ```)
        pattern1 = r'```json\s*\n(.*?)\n```'
        matches1 = re.findall(pattern1, last_text, re.DOTALL)

        if matches1:
            json_block = matches1[0]
        else:
            # Fallback: basic {...} block
            pattern2 = r'\{[^{}]*\}'
            matches2 = re.findall(pattern2, last_text, re.DOTALL)

            if matches2:
                json_block = matches2[0]
            else:
                print(f"❌ No JSON found for row {i}")
                continue

        fixed = fix_json_with_library(json_block)

        # ✅ Store metadata alongside fixed JSON
        all_fixed_jsons.append({
            "row_index": i,
            "company_id": row.get("company_id", "Unknown"),
            "year": row.get("year", "Unknown"),
            "fixed_json": fixed
        })

    return all_fixed_jsons


In [71]:
# import re

# def fixed_json(df):
#     all_fixed_jsons = []

#     for i, row in df.iterrows():
#         predicted = str(row.predicted_direction).strip().lower()
        
#         # Skip rows that are already correct
#         if predicted in ["increase", "decrease"]:
#             continue

#         full_response_text = row.full_model_response

#         # If it contains the JSON key, extract everything after that
#         if "direction_of_eps_change" in full_response_text:
#             # Grab everything from the first occurrence of that key
#             start_index = full_response_text.find("direction_of_eps_change")
#             json_like_text = full_response_text[start_index:].strip()

#             all_fixed_jsons.append(json_like_text)
#             continue

#         # If not even the key is present, try regex-based extraction
#         last_text = full_response_text.split('assistant')[-1]

#         # First pattern: ```json ... ```
#         pattern1 = r'```json\s*\n(.*?)\n```'
#         matches1 = re.findall(pattern1, last_text, re.DOTALL)

#         if matches1:
#             json_block = matches1[0]
#         else:
#             # Fallback: simple { ... } block
#             pattern2 = r'\{[^{}]*\}'
#             matches2 = re.findall(pattern2, last_text, re.DOTALL)
#             if matches2:
#                 json_block = matches2[0]
#             else:
#                 print(f"No JSON found for row {i}")
#                 continue  # Skip if nothing is found

#         # Optional: clean or fix JSON here if needed
#         fixed = fix_json_with_library(json_block)
#         all_fixed_jsons.append(fixed)

#     return all_fixed_jsons


In [72]:
rows = fixed_json(eval_df)
print(len(rows))
# print(rows)
print(rows[0])

560
{'row_index': 4, 'company_id': 1000, 'year': 1977, 'fixed_json': {'direction_of_eps_change': 'Moderate Increase', 'magnitude_of_change': 'Dependent on successful execution of growth initiatives while optimizing cost structures and managing existing debts.', 'certainty_of_assessment': 'Medium', 'reason': '\\nThe company’s demonstrated ability to grow revenues with expanding markets alignss with upward pressure on earnings per share. However, the balance between financial leverage and operational efficiency remains critical, affecting both the extent of such increase and its predictability.'}}


In [95]:
import re

def process_non_error_rows(eval_df):
    all_fixed_jsons = []

    # Filter rows where predicted_direction is not error/increase/decrease
    filtered_df = eval_df[
        ~eval_df['predicted_direction'].str.lower().isin(['error', 'increase', 'decrease'])
    ]

    for i, row in filtered_df.iterrows():
        # print(row)
        company_id = row.get("company_id", "Unknown")
        year = row.get("year", "Unknown")
        full_response_text = row.full_model_response

        # Extract text after the second occurrence of 'assistant'
        full_response_text = row.full_model_response
        last_text = full_response_text.split('assistant')[-1]

        # Look for JSON block inside ```json ... ```
        pattern1 = r'```json\s*\n(.*?)\n```'
        matches1 = re.findall(pattern1, last_text, re.DOTALL)

        if matches1:
            json_block = matches1[0]
        else:
            # Fallback: basic {...} block
            pattern2 = r'\{[^{}]*\}'
            matches2 = re.findall(pattern2, last_text, re.DOTALL)

            if matches2:
                json_block = matches2[0]
            else:
                print(f"❌ No JSON found for row {i}")
                print(row)
                continue

        fixed = fix_json_with_library(json_block)

        # ✅ Store metadata alongside fixed JSON
        all_fixed_jsons.append({
            "row_index": i,
            "company_id": row.get("company_id", "Unknown"),
            "year": row.get("year", "Unknown"),
            "fixed_json": fixed
        })

    return all_fixed_jsons


In [96]:
non_error_fixed = process_non_error_rows(eval_df)
print(len(non_error_fixed))

❌ No JSON found for row 616
company_id                                                          1510
year                                                                1976
actual_label                                                    increase
predicted_direction                                  <increase/decrease>
magnitude              <Brief description of magnitude, e.g., Slight,...
certainty                                              <High/Medium/Low>
reason                 <Concise explanation for the prediction. Ensur...
is_match                                                           False
full_model_response    system\nYou are a helpful assistant.\nuser\nBa...
Name: 616, dtype: object
❌ No JSON found for row 755
company_id                                                          1608
year                                                                1973
actual_label                                                    decrease
predicted_direction                        

In [77]:
from transformers import pipeline

def zero_shot(rows):
    # Load zero-shot classifier
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    predictions = []

    for idx, row in enumerate(rows):
        print(f"🔍 Row {idx + 1}")

        # Try reason first
        reason = row.get('fixed_json', {}).get('reason', '').strip()
        fallback = row.get('fixed_json', {}).get('direction_of_eps_change', '').strip()

        if reason:
            text_to_classify = reason
            source = "reason"
        elif fallback:
            text_to_classify = fallback
            source = "direction_of_eps_change"
        else:
            print(f"⚠️ Skipping row {idx}: No text to classify.")
            predictions.append({
                "row_index": row.get("row_index"),
                "company_id": row.get("company_id"),
                "year": row.get("year"),
                "source": "none",
                "text": "",
                "predicted_label": "Unknown"
            })
            continue

        try:
            result = classifier(
                text_to_classify,
                candidate_labels=["Increase", "Decrease"],
            )
            predicted_label = result["labels"][0]

        except Exception as e:
            print(f"❌ Error during classification on row {idx}: {str(e)}")
            predicted_label = "Error"

        predictions.append({
            "row_index": row.get("row_index"),
            "company_id": row.get("company_id"),
            "year": row.get("year"),
            "source": source,
            "text": text_to_classify,
            "predicted_label": predicted_label
        })

        print(f"✅ Used {source}: {text_to_classify[:60]}... → {predicted_label}")
        print("-" * 80)

    return predictions


In [ ]:
# non_error_fixed = process_non_error_rows(eval_df)
# print(len(non_error_fixed))

predictions = zero_shot(non_error_fixed)
len(predictions)

Device set to use mps:0


🔍 Row 1
✅ Used reason: \nThe firm's expanding business scale benefits its positioni... → Increase
--------------------------------------------------------------------------------
🔍 Row 2
✅ Used reason: Despite notable revenue growth, increasing operating expense... → Decrease
--------------------------------------------------------------------------------
🔍 Row 3
✅ Used reason: \e Since the company has shown signs of improving operationa... → Increase
--------------------------------------------------------------------------------
🔍 Row 4
✅ Used reason: \nPotential for increased profitability comes mainly from st... → Increase
--------------------------------------------------------------------------------
🔍 Row 5
✅ Used reason: \nThe predicted stabilization or minor uptick in EPS hinges ... → Increase
--------------------------------------------------------------------------------
🔍 Row 6
✅ Used reason: \e*Revenue growth\e+ suggests healthy demand; \e*Improved gr... → Increase
-------

276

In [120]:
import pandas as pd

pred_df = pd.DataFrame(predictions)
pred_df.head()
pred_df['predicted_label'].value_counts()


predicted_label
Increase    150
Decrease    100
Unknown      26
Name: count, dtype: int64

In [101]:
# Make sure the predictions list is the same length as the DataFrame rows
rows = fixed_json(eval_df)
error_predictions = zero_shot(rows)



Device set to use mps:0


🔍 Row 1
✅ Used reason: \nThe company’s demonstrated ability to grow revenues with e... → Increase
--------------------------------------------------------------------------------
🔍 Row 2
✅ Used reason: Considering the moderate financial performance indicators an... → Decrease
--------------------------------------------------------------------------------
🔍 Row 3
✅ Used reason: \eThis assessment is based on the observed trend towards inc... → Decrease
--------------------------------------------------------------------------------
🔍 Row 4
✅ Used reason: \e1\e2Increasing revenues along with controlled costs offer ... → Increase
--------------------------------------------------------------------------------
🔍 Row 5
✅ Used reason: The company has shown substantial improvement in revenues an... → Increase
--------------------------------------------------------------------------------
🔍 Row 6
✅ Used direction_of_eps_change: Moderate Increase... → Increase
---------------------------------

In [119]:
error_df = pd.DataFrame(error_predictions)
error_df.head()
error_df['predicted_label'].value_counts()

predicted_label
Increase    328
Decrease    232
Name: count, dtype: int64

In [104]:
combined_preds = pd.concat([pred_df, error_df], ignore_index=True)
len(combined_preds)

836

In [105]:
combined_preds['company_id'] = combined_preds['company_id'].astype(str)
combined_preds['year'] = combined_preds['year'].astype(str)
eval_df['company_id'] = eval_df['company_id'].astype(str)
eval_df['year'] = eval_df['year'].astype(str)


In [115]:
# Step 1: Combine pred_df and error_df
combined_preds = pd.concat([pred_df, error_df], ignore_index=True)

# Step 2: Standardize types
combined_preds['company_id'] = combined_preds['company_id'].astype(str)
combined_preds['year'] = combined_preds['year'].astype(str)
eval_df['company_id'] = eval_df['company_id'].astype(str)
eval_df['year'] = eval_df['year'].astype(str)

# Step 3: Create a lookup dictionary {(company_id, year): predicted_label}
pred_lookup = {
    (row['company_id'], row['year']): row['predicted_label']
    for _, row in combined_preds.iterrows()
}

# Step 4: Update eval_df in-place
def update_prediction(row):
    key = (row['company_id'], row['year'])
    return pred_lookup.get(key, row['predicted_direction'])

eval_df['predicted_direction'] = eval_df.apply(update_prediction, axis=1)


In [ ]:
eval_df.head()


,company_id,year,actual_label,predicted_direction,magnitude,certainty,reason,is_match,full_model_response,predicted_label
0,1000,1973,increase,decrease,Significant,Medium,\eSince\e\ sales\ehave\edeclined\esharply\ewit...,False,system\nYou are a helpful assistant.\nuser\nBa...,NaN
1,1000,1974,decrease,decrease,Moderate,Medium,"Despite some growth signals via asset buildup,...",True,system\nYou are a helpful assistant.\nuser\nBa...,NaN
2,1000,1975,increase,increase,Moderate,High,Rapidly increasing sales coupled with margin e...,True,system\nYou are a helpful assistant.\nuser\nBa...,NaN
3,1000,1976,increase,decrease,Moderate,Medium,\nThe projected decrease in EPS stems from ant...,False,system\nYou are a helpful assistant.\nuser\nBa...,NaN
4,1000,1977,decrease,Increase,Error,Error,"Error in parsing JSON: Expecting ',' delimiter...",False,system\nYou are a helpful assistant.\nuser\nBa...,Increase


In [123]:
eval_df['predicted_direction'].value_counts()


predicted_direction
increase               559
Increase               478
decrease               398
Decrease               332
<increase/decrease>      7
Name: count, dtype: int64

In [128]:
# Step 1: Standardize text
eval_df['predicted_direction'] = eval_df['predicted_direction'].astype(str).str.strip().str.lower()
eval_df['actual_label'] = eval_df['actual_label'].astype(str).str.strip().str.lower()

# Step 2: Drop rows where predicted_direction == "<increase/decrease>"
eval_df = eval_df[eval_df['predicted_direction'] != "<increase/decrease>"]

# Step 3: Drop rows where actual_label == "neutral"
eval_df = eval_df[eval_df['actual_label'] != "neutral"]


In [129]:
print(eval_df['predicted_direction'].value_counts())


predicted_direction
increase    1026
decrease     722
Name: count, dtype: int64


In [130]:
eval_df['actual_label'].value_counts()

actual_label
increase    1172
decrease     576
Name: count, dtype: int64

In [131]:
from sklearn.metrics import f1_score

# Make sure both columns are lowercase and stripped
eval_df['actual_label'] = eval_df['actual_label'].astype(str).str.strip().str.lower()
eval_df['predicted_direction'] = eval_df['predicted_direction'].astype(str).str.strip().str.lower()

# Compute F1 score (binary classification: 'increase' vs 'decrease')
f1 = f1_score(eval_df['actual_label'], eval_df['predicted_direction'], pos_label='increase', average='binary')
print(f"🔍 F1 Score (for label='increase'): {f1:.4f}")


🔍 F1 Score (for label='increase'): 0.6151
